# Prepare Diva

this notebooks prepares the version of the diva dataset I'll work on

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from pathlib import Path
import os
import sys
import category_encoders as ce
import bz2
import pickle
np.random.seed(10)
# adds the visibility of the mlem module, needed to load the attack models
sys.path.append("../../../../") 
import mlem
from sklearn.preprocessing import StandardScaler
from mlem.utilities import load_pickle_bz2, save_pickle_bz2, sample_from_quantile

In [2]:
df = pd.read_excel("../processed_diva.xlsx")
df = df.dropna()
df = df.drop_duplicates()
features, target = df.iloc[:,:-1], df.iloc[:,-1]
target = target.map({0.0: 0, 3.0: 1})
numeric = features.select_dtypes('number')

In [3]:
FEATURES = ['FLG_PRES_RISCOSS',
 'IMP_V_AGG_IVA',
 'SOLVIBILITA',
 'VOL_AFF_DICH',
 'PESO_ADESIONE',
 'TIPO_DICH_ACCERT',
 'IMP_ESISTZ',
 'DETR_IVA_ACC',
 'VAL_ALIQ_MEDIA_ACQ',
 'FLG_PRES_RICORSO',
 'STATO_CONTROLLO',
 'VAL_ALIQ_M_ACQ_IMP',
 'FLG_VC',
 'IMP_V_AGG_IMPON',
 'VAR_RIMBORSO',
 'VAL_ALIQ_MEDIA_VOL',
 'IMP_IMPST_CREDIT',
 'IMP_ACQ_NOIMP',
 'COD_ATTIV_GEN',
 'IVA_OP_IMPON_DIC',
 'IMP_BEN_AMM',
 'IMP_ECC_PREC',
 'FLG_PRES_BILANCIO',
 'imp_tot_pos',
 'VAR_DETRAZIONE',
 'MAG_IMP_RIT_ACC']

In [4]:
df = df[FEATURES].copy()
df['Target'] = target
df

,FLG_PRES_RISCOSS,IMP_V_AGG_IVA,SOLVIBILITA,VOL_AFF_DICH,PESO_ADESIONE,TIPO_DICH_ACCERT,IMP_ESISTZ,DETR_IVA_ACC,VAL_ALIQ_MEDIA_ACQ,FLG_PRES_RICORSO,...,IMP_ACQ_NOIMP,COD_ATTIV_GEN,IVA_OP_IMPON_DIC,IMP_BEN_AMM,IMP_ECC_PREC,FLG_PRES_BILANCIO,imp_tot_pos,VAR_DETRAZIONE,MAG_IMP_RIT_ACC,Target
0,1.0,5306335.0,3.0,27245028.00,48.0,3.0,0.0,1337486.0,6.59,0.0,...,428791.0,6.0,-466745.0,61681.0,10728.0,1.0,0.0,466745.0,0.0,0
1,0.0,172597.0,3.0,1283657.75,0.0,2.0,0.0,0.0,3.44,0.0,...,6227.0,10.0,-38741.5,20968.0,0.0,0.0,0.0,4.0,0.0,0
2,0.0,291567.0,3.0,1247396.00,0.0,3.0,0.0,0.0,11.35,0.0,...,0.0,8.0,-48344.0,3716.0,8799.0,1.0,0.0,48344.0,0.0,0
3,0.0,-13314.0,4.0,21636.00,0.0,3.0,0.0,0.0,18.40,0.0,...,0.0,12.0,-8625.0,0.0,0.0,1.0,0.0,8625.0,0.0,0
4,1.0,20237.0,3.0,765720.00,0.0,2.0,0.0,0.0,6.10,0.0,...,0.0,10.0,-10329.0,15434.0,0.0,0.0,0.0,10329.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12494,0.0,-1778240.0,3.0,0.00,0.0,3.0,0.0,263885.0,19.67,0.0,...,0.0,8.0,-524045.0,1744944.0,0.0,0.0,0.0,524045.0,0.0,1
12495,0.0,-24041.0,3.0,0.00,0.0,3.0,0.0,0.0,20.00,0.0,...,0.0,16.0,-69870.0,0.0,259.0,0.0,0.0,69870.0,0.0,1
12496,0.0,-447022.0,3.0,0.00,0.0,3.0,0.0,0.0,19.99,0.0,...,0.0,18.0,-108968.0,445185.0,6.0,1.0,0.0,42968.0,0.0,1
12497,0.0,-40617.0,3.0,0.00,0.0,3.0,0.0,0.0,19.84,0.0,...,0.0,18.0,-23087.0,39960.0,0.0,1.0,0.0,23087.0,0.0,1


In [5]:
scaler = StandardScaler()
df[FEATURES] = scaler.fit_transform(df[FEATURES])
df

,FLG_PRES_RISCOSS,IMP_V_AGG_IVA,SOLVIBILITA,VOL_AFF_DICH,PESO_ADESIONE,TIPO_DICH_ACCERT,IMP_ESISTZ,DETR_IVA_ACC,VAL_ALIQ_MEDIA_ACQ,FLG_PRES_RICORSO,...,IMP_ACQ_NOIMP,COD_ATTIV_GEN,IVA_OP_IMPON_DIC,IMP_BEN_AMM,IMP_ECC_PREC,FLG_PRES_BILANCIO,imp_tot_pos,VAR_DETRAZIONE,MAG_IMP_RIT_ACC,Target
0,1.518260,0.230849,0.228899,0.093718,4.058525,0.477425,-0.082859,0.161744,-1.833110,-0.542534,...,0.028205,-0.761774,-0.275266,-0.025103,-0.012631,0.828543,-0.139304,0.531387,-0.035737,0
1,-0.658649,-0.008665,0.228899,-0.043983,-0.449675,-1.161895,-0.082859,-0.077551,-2.448218,-0.542534,...,-0.035253,0.158846,0.151410,-0.025933,-0.075926,-1.206937,-0.139304,-0.215376,-0.035737,0
2,-0.658649,-0.003115,0.228899,-0.044175,-0.449675,0.477425,-0.082859,-0.077551,-0.903613,-0.542534,...,-0.036188,-0.301464,0.141838,-0.026284,-0.024012,0.828543,-0.139304,-0.138035,-0.035737,0
3,-0.658649,-0.017339,1.194882,-0.050677,-0.449675,0.477425,-0.082859,-0.077551,0.473058,-0.542534,...,-0.036188,0.619156,0.181433,-0.026360,-0.075926,0.828543,-0.139304,-0.201583,-0.035737,0
4,1.518260,-0.015774,0.228899,-0.046730,-0.449675,-1.161895,-0.082859,-0.077551,-1.928793,-0.542534,...,-0.036188,0.158846,0.179735,-0.026045,-0.075926,-1.206937,-0.139304,-0.198857,-0.035737,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12494,-0.658649,-0.099681,0.228899,-0.050791,-0.449675,0.477425,-0.082859,-0.030338,0.721054,-0.542534,...,-0.036188,-0.301464,-0.332388,0.009194,-0.075926,-1.206937,-0.139304,0.623064,-0.035737,1
12495,-0.658649,-0.017839,0.228899,-0.050791,-0.449675,0.477425,-0.082859,-0.077551,0.785494,-0.542534,...,-0.036188,1.539776,0.120378,-0.026360,-0.074398,-1.206937,-0.139304,-0.103594,-0.035737,1
12496,-0.658649,-0.037574,0.228899,-0.050791,-0.449675,0.477425,-0.082859,-0.077551,0.783542,-0.542534,...,-0.036188,2.000086,0.081402,-0.017289,-0.075890,0.828543,-0.139304,-0.146636,-0.035737,1
12497,-0.658649,-0.018613,0.228899,-0.050791,-0.449675,0.477425,-0.082859,-0.077551,0.754251,-0.542534,...,-0.036188,2.000086,0.167016,-0.025546,-0.075926,0.828543,-0.139304,-0.178445,-0.035737,1


In [8]:
df.Target.value_counts(normalize=True)

0    0.737679
1    0.262321
Name: Target, dtype: float64

In [7]:
df.to_csv("diva_clean.csv", index=False)